# Agents for Amazon Bedrock - Testing Agent Invocation

This notebook provides sample code for testing the agent invocation using the [boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime.html) client for `bedrock-agent-runtime`

### Use Case
Now that we've created the agent and attached the knowledge base to it, we will use the runtime client to invoke the agent with different queries. The boto3 sdk for agent is divided into two clients: `bedrock-agent` and `bedrock-agent-runtime`. The `bedrock-agent` client is responsible for the functionalities to create, update, delete and/or prepare an agent or a knowledge base. While the `bedrock-agent-runtime` is responsible for invoke an agent (with the [`invoke_agent`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/invoke_agent.html) API) and retrieve documents from a knowledge base (with the [`retrieve`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve.html) and [`retrieve_and_generate`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve_and_generate.html) APIs). This notebook will focus on the runtime invocation of the agent created in the previous notebooks. We will use the `invoke_agent` API.



### Notebook Walkthrough

In this notebook we will:
- Retrieve the saved variables from the previous notebook
- Invoke agent with a new session using Knowledge Bases and Action Groups
- Invoke agent with an existent session
- Invoke agent with prompt attributes
- Invoke agent with trace enabled
- Invoke agent with different languages for the request


### Next Steps: 
In the next lab, we will clean up the resources created

### Pre-requisites

Before starting this lab, we need to load the variables that we stored in the previous notebook.

In [1]:
%store -r

#### Importing required packages and initiate variables

Let's also import the necessary packages, setup the logging and initiate the boto3 client for `bedrock-agent-runtime`. We are also initiating the client for `dynamodb` as a support functionality to check that the tasks were performed correctly by our agent

In [2]:
import boto3
import logging
import pprint
import json
import pandas as pd
from agent import invoke_agent_helper
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')
dynamodb = boto3.resource('dynamodb')

#### Create support invokeAgent function

Let's again use the same support function called `invoke_agent_helper` from `agents.py` to allow us to invoke the agent with or without trace enabled and with or without session state. 

This function allows the user to send a `query` to the agent with a `session_id`. The user can then decide to enable trace or not using the `enable_trace` boolean variable and to pass a session state as a dictionary via the `session_state` variable.

If a new `session_id` is provided, the agent will create a new conversation without previous context. If the same `session_id` is reused, the conversation history related to that session id is made available to the agent.

If the `enable_trace` is set to `True`, each response from the agent is accompanied by a *trace* that details the step being orchestrated by the agent. It allows you to follow the agent's (reasoning via Chain of Thoughts prompting) that led to the final response at that point of the conversation.

Finally, you can also pass a session context using the `session_state` parameter. The session state allows you to share the following information with the agent:
- **`sessionAttributes`**: attributes that persist over a session between the user and the agent. All invokeAgent calls with the same session_id belong to the same sesison and will have the sessionAttributes shared with them as long as the session time limit has not being surpassed and the user has not ended the session. The sessionAttributes are available in the lambda function but are **not** added to the agent's prompt. As a result, you can only use session attributes if your lambda function can handle them. You can find more examples of using a session attribute [here](https://github.com/aws-samples/amazon-bedrock-samples/tree/main/agents-for-bedrock/features-examples/06-prompt-and-session-attributes). It is also a good pattern to implement fine-grained access control for certain APIs using the lambda function integration. You can find an example for it [here](https://github.com/aws-samples/amazon-bedrock-samples/tree/main/agents-for-bedrock/features-examples/09-fine-grained-access-permissions)
- **`promptSessionAttributes`**: attributes that persist over a single invokeAgent call. Prompt attributes are added to the prompt and to the lambda function. You can also use the `$prompt_session_attributes$` placeholder when editing the orchestration base prompt.
- **`invocationId`**: The id returned by the agent in the [ReturnControlPayload](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_agent-runtime_ReturnControlPayload.html) object in the returnControl field of the InvokeAgent response. This field is required if passing the answer of a Return of Control invocation. You can find an example of how to use it [here](https://github.com/aws-samples/amazon-bedrock-samples/tree/main/agents-for-bedrock/features-examples/03-create-agent-with-return-of-control).
- **`returnControlInvocationResults`**: the results obtained from invoking the action outside of agents for Amazon Bedrock.  This field is required if passing the answer of a Return of Control invocation. You can find an example of how to use it [here](https://github.com/aws-samples/amazon-bedrock-samples/tree/main/agents-for-bedrock/features-examples/03-create-agent-with-return-of-control).

#### Create support selectAllFromDynamoDB function

We will also create the support function called `selectAllFromDynamoDB` to select all data in the dynamoDB table `restaurant_bookings`. This function will be used to validate our agent's behaviour

In [3]:
def selectAllFromDynamodb():
    # Get the table object
    table = dynamodb.Table(table_name)

    # Scan the table and get all items
    response = table.scan()
    items = response['Items']

    # Handle pagination if necessary
    while 'LastEvaluatedKey' in response:
        response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
        items.extend(response['Items'])

    items = pd.DataFrame(items)
    return items

In [4]:
# test function invocation
items = selectAllFromDynamodb()
items

""


### Invoking agent with a new session id
Let's first use the `InvokeAgent` function to query the Knowledge Base with the agent without previous context

### Invoking agent with existent session id

Next we can use the context to ask a follow up question. To do so, we use the same `session_id` with the `invokeAgent` function

In [5]:
%%time
import uuid
session_id:str = str(uuid.uuid1())

query = "what its check-in and checkout time?"
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)



Check-in is at 3:00 PM and check-out is at 11:00 AM local time. Early check-in and late check-out are subject to availability and may incur additional charges.


 
CPU times: user 10.6 ms, sys: 0 ns, total: 10.6 ms
Wall time: 4.7 s


As you can see, the agent knows that we are talking about the children's menu.

### Invoking agent using action group

Now let's use our agent to make a reservation. By doing so, we will require the agent to execute an action from our action group to create a new reservation.

In [9]:
%%time
import uuid
query = """I want to book a room in ABC grand hotel, but 
I know my booking means I am investing in it. So lets first find out
how has revenue grown from 2022 to 2023 for ABC Grand Hotel? And What's the annual growth rate?"""
session_id:str = str(uuid.uuid1())

response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)



The combined total revenue of hotels under management increased from US$1,568.1 million in 2022 to US$1,890.2 million in 2023, a growth of 21%.





The annual growth rate is 21%.




CPU times: user 10.8 ms, sys: 0 ns, total: 10.8 ms
Wall time: 5.38 s


In [10]:
%%time
query = "Awesome! I think thats postive. Can you create a booking for 4 people under name 'Reinvent Booking' with a checkin at 9pm on the 5th of May 2024."
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)

Your booking has been successfully created with the booking ID 'd88628f3'. You will check in at 9pm on the 5th of May 2024 under the name 'Reinvent Booking' for 4 people.
CPU times: user 6.8 ms, sys: 3.87 ms, total: 10.7 ms
Wall time: 6.06 s


Let's double check that the data was properly added to the dynamoDB table

In [11]:
selectAllFromDynamodb()

,num_guests,date,hour,booking_id,name
0,4,2024-05-05,21:00,d88628f3,Reinvent Booking


Great! We've used our agent to do create a reservation. However, often when booking rooms in hotel we are already logged in to a system that know our names. How great would it be if our agent would know it as well?

To do so, we can use the session context to provide some attributes to our prompt. In this case we will provide it directly to the prompt using the [promptSessionAttributes](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-session-state.html) parameter. Let's also start a new session id so that our agent does not memorize our name.

In [15]:
%%time
session_id:str = str(uuid.uuid1())
session_state = {
    "promptSessionAttributes": {
        "Nombre": "Reinvent Booking for 4 people at 9pm on the 5th of May 2024"
    }
}
query = "I want to create a booking for 2 people, at 8pm on the 6th of May 2024, with name Reinvent Booking Login"
response = invoke_agent_helper(query, session_id, agent_id, alias_id, session_state=session_state)
print(response)

Your booking has been created successfully. Your booking ID is 'ef180d6f'. 
CPU times: user 6.89 ms, sys: 3.76 ms, total: 10.6 ms
Wall time: 4.12 s


Again let's validate the the correct data was added to dynamoDB

In [16]:
selectAllFromDynamodb()

,num_guests,date,hour,booking_id,name
0,4,2024-05-05,21:00,d88628f3,Reinvent Booking
1,2,2024-05-06,20:00,2c0f20a6,Reinvent Booking Login
2,2,2024-12-01,19:30,39218bac,Gabierela
3,2,2024-05-06,20:00,ef180d6f,Reinvent Booking Login


We can also validate the data using our agent's session information since the agent knows the booking id to invoke the get booking details function

In [17]:
%%time
query = "Get the details for the last booking created"
booking_id = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(booking_id)

Here are the details of your last booking: Number of guests: 2, Date: 2024-05-06, Time: 20:00, Booking ID: ef180d6f, Name: Reinvent Booking Login.
CPU times: user 7.87 ms, sys: 3.12 ms, total: 11 ms
Wall time: 4.1 s


#### Deleting booking created

Let's also test the delete booking functionality by deleting the last created booking id

In [18]:
%%time
query = f"I want to delete the booking."
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)

Your booking has been deleted successfully.
CPU times: user 10 ms, sys: 225 μs, total: 10.2 ms
Wall time: 3.28 s


And we confirm that the booking has also been deleted from the dynamoDB table

In [19]:
selectAllFromDynamodb()

,num_guests,date,hour,booking_id,name
0,4,2024-05-05,21:00,d88628f3,Reinvent Booking
1,2,2024-05-06,20:00,2c0f20a6,Reinvent Booking Login
2,2,2024-12-01,19:30,39218bac,Gabierela


#### Invoking agent using promptSessionAttributes to handle temporal information

With real-life applications, context is really important. We want to make reservations considering the current date and the days sorounding it. Agents for Amazon Bedrock also allow you to provide temporal context for the agent with the prompt attributes. Let's test it with a reservation for tomorrow

In [20]:
# retrieving today
from datetime import datetime
today = datetime.today().strftime('%b-%d-%Y')
today

'Feb-07-2025'

In [21]:
%%time
# reserving a table for tomorrow
session_id:str = str(uuid.uuid1())
name = "Reinvent Booking Login"
query = f"""I want to create a booking for 2 people, at 8pm tomorrow, 
name of the booking is {name}, date is {today}"""
response = invoke_agent_helper(query, session_id, agent_id, alias_id, session_state=session_state)
print(response)

Your booking has been created successfully. Your booking ID is 'c241e898'. 
CPU times: user 11.1 ms, sys: 0 ns, total: 11.1 ms
Wall time: 4.44 s


And now to confirm that everything got added properly, let's retrieve the details of the last booking 

In [22]:
%%time
query = "Could you get the details for the last booking created?"
booking_id = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(booking_id)

The details of your last booking are as follows: Booking ID: c241e898, Name: Reinvent Booking Login, Date: 2025-02-07, Time: 20:00, Number of Guests: 2.
CPU times: user 10.3 ms, sys: 0 ns, total: 10.3 ms
Wall time: 4.22 s


#### Asking the agent for other recomendations

Another good use case for our agent, is to use its reasoning capabilities to ask for some other recommendation. Let's look at a couple of examples for it

In [23]:
%%time
session_id:str = str(uuid.uuid1())
query = "Are there any Hair cut ammenities in ABC Grand?"
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)



Room amenities include: Hair dryers, irons and ironing boards. Coffee makers are not included, but coffee delivery can be arranged through Room Service. All rooms offer High-Speed Internet access. Taxi and Rideshare services are available. The Concierge can assist with booking shuttle or Limousine transportation. Electric car charging stations are available on the 1st floor of the self parking garage, first come first served. To add additional names to a reservation, contact ABC Hotel Reservations or the Front Desk Team. Contact the Concierge to arrange special occasion amenities. Check-in is at 3:00 PM and check-out is at 11:00 AM local time. Early check-in and late check-out are subject to availability and may incur additional charges. The ABC Rewards Mobile App offers features like mobile check-in/out. ABC Rewards members can check in early and be notified when their room is ready. Room charges can be used to redeem credits. See details on where to use credits on the hotel website

### Invoking agent with trace enabled

We can also invoke our agent with the trace enabled to produce the details of each step being orchestrated by the Agent. Let's see all the steps executed when checking for documents in the knowledge base

In [24]:
%%time
session_id:str = str(uuid.uuid1())
query = "What is the security deposite, and is parking provided at ABC Hotel ?"
response = invoke_agent_helper(query, session_id, agent_id, alias_id, enable_trace=True)
print(response)

[2025-02-07 11:50:01,435] p17318 {agent.py:176} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/vnd.amazon.eventstream',
                                      'date': 'Fri, 07 Feb 2025 11:50:01 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': 'a9ad05f6-e549-11ef-8f5d-6e38dbb67380',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': '1edd4768-e1e7-414d-86eb-d35f9427346c'},
                      'HTTPStatusCode': 200,
                      'RequestId': '1edd4768-e1e7-414d-86eb-d35f9427346c',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x7f42777c3490>,
 'contentType': 'application/json',
 'sessionId': 'a9ad05f6-e549-11ef-8f5d-6e38dbb67380'}


[2025-02-07 11:50:01,645] p17318 {agent.py:190} INFO - {
  "agentAliasId": "TSTALIASID",
  "agentId": "WOTFRWMOH8",
  "agentVersion": "DRAFT",
  "callerChain": [
    {
      "agentAliasArn": "arn:aws:bedrock:us-west-2:710299592439:agent-alias/WOTFRWMOH8/TSTALIASID"
    }
  ],
  "sessionId": "a9ad05f6-e549-11ef-8f5d-6e38dbb67380",
  "trace": {
    "orchestrationTrace": {
      "modelInvocationInput": {
        "inferenceConfiguration": {
          "maximumLength": 1024,
          "stopSequences": [
            "</answer>",
            "\n\n<thinking>",
            "\n<thinking>",
            " <thinking>"
          ],
          "temperature": 1.0,
          "topK": 1,
          "topP": 1.0
        },
        "text": "{\"system\":\"Agent Description:You are a ABC Grand Hotel Booking agent, one of the key chain of hotels Hosting Amazon Reinvent 2024 helping clients retrieve information about the hotel,create a new booking or cancel an existing bookingAlways follow these instructions:- Do 



A security deposit of $100 per day is required upon check-in, with a maximum of $400 for credit/debit cards. For complete details on parking, please visit the ABC Hotel parking information page.


 
CPU times: user 23.2 ms, sys: 0 ns, total: 23.2 ms
Wall time: 4.51 s


### Invoking agent with different prompt languages

The last feature that we will highlight in this notebook is the ability of LLM models to handle inputs in different languages. We've only implemented our agent in English, but wouldn't it be great if it could also handle other languages?

Good news is that it can! This is due to the LLM multi-language capabilities. And the best part is that we don't need to change anything in the agent and it will even going to respond in the requested language.

Let's try a couple of queries!

![images/agent/language_support.png](images/agent/language_support.png)

In [25]:
%%time
# Invoking agents in spanish
session_id:str = str(uuid.uuid1())
query = "¿Podría reservar una habitación para dos el 1/12/2024 con un registro a las 7:30 p. m. en Reinvent Booking for Gabierela?"
session_state = {
    "promptSessionAttributes": {
        "Nombre": "Reinvent Booking for Gabierela"
    }
}
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)

¡Reserva realizada con éxito! Su reserva para Gabierela ha sido confirmada para el 1/12/2024 con un registro a las 7:30 p. m. Su ID de reserva es 5cad80fb.

CPU times: user 10.1 ms, sys: 233 μs, total: 10.3 ms
Wall time: 5.28 s



![images/agent/result_translation.png](images/agent/result_translation.png)

Last, let's check our dynamoDB to see all the bookings available

In [26]:
selectAllFromDynamodb()

,num_guests,date,hour,booking_id,name
0,4,2024-05-05,21:00,d88628f3,Reinvent Booking
1,2,2024-05-06,20:00,2c0f20a6,Reinvent Booking Login
2,2,2025-02-07,20:00,c241e898,Reinvent Booking Login
3,2,2024-12-01,19:30,5cad80fb,Gabierela
4,2,2024-12-01,19:30,39218bac,Gabierela


### BONUS Round!

**Party Rock Bonus Round**
Unleash your creativity, Mix and match Nova Understanding models with Nova Canvas model to create something useful and meaningful. Based on creativity, relevancy and usefulness top 3 winners will be awarded 25 USD Amazon Gift Card (Only Applicable to candidates residing within US)